This file details our efforts to compare a variety of different prompts and parameters in the use of a pretrained model to conduct few shot learning and generate a fake profile that sounds indistinguishable from other top matches.

Different avenues explored include using:
    - Full essays 
        - Example prompt:
            Input: 
            Output:
            Input:
            Output:
            Input:
            
    - First x number of characters of an essay
            Example prompt: Same as above but cut off each essay after x characters
            
    - Good and bad match
        - Example prompt:
            This is the input essay: 
            This is a good match:
            This is a bad match: 
            Write a new good match:
            
    - Experiment with different parameters of the generate function
    
    - different/larger versions of GPT - implement when we have more computational resources



**NEED TO UPDATE This file loads in a pretrained model from the HuggingFace library, and uses most of an individuals essays and a csv containing the indices of their top match, to construct a prompt for a few shot encoder to produce a fake profile in the same style/tone that should also be a good match for them.




In [11]:
!pip install sentence_transformers
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import pandas as pd
from scipy.spatial.distance import cosine

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/68/0d/27475988a3daade7516ea02dbc607b57d4a30f01bb49614a6430e76685c2/sentence_transformers-2.5.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.4 MB/s eta 0:00:00 MB/s eta 0:00:01m


In [12]:
model = SentenceTransformer('all-MiniLM-L6-v2')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
def rank_new_input(input_str, pref_gender=False, pref_age_lower=False, pref_age_higher=False, min_similarity_score = 0.5):
    df_possible = df_all.copy()
    if pref_gender:
        df_possible = df_possible.loc[df_possible.loc[:,'sex'] == pref_gender, :]
    if pref_age_higher:
        df_possible = df_possible[df_possible.loc[:, "age"] <= pref_age_higher]
    if pref_age_lower:
        df_possible = df_possible[df_possible.loc[:, "age"] >= pref_age_lower]
    user_embeddings = model.encode(input_str)
    other_embeddings = [embedding_series[i] for i in df_possible.index]
    
    # Compute the cosine similarity between the user's weighted embedding vector and all possible matches
    cosine_similarities = compute_cosine_similarity(user_embeddings, other_embeddings)
    # Recover index to match back to original dataframe
    similarity_scores = [(df_possible.index[index], score) for index, score in enumerate(cosine_similarities) if score >= min_similarity_score and score != 1]
    # Sort by similarity
    ranked_similarity = sorted(similarity_scores, key = lambda x: x[1], reverse = True)
    return ranked_similarity

In [14]:
df_all = pd.read_csv('./okcupid_profiles.csv')
df_all

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,...,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,...,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",...,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59,single,f,straight,NaN,NaN,socially,never,graduated from college/university,NaN,...,"vibrant, expressive, caring optimist. i love b...",the happiest times have been when life came to...,i make an outstanding osso bucco. i am also ve...,"i am told that people notice my smile, eyes an...",i am an avid movie watcher and follow the broa...,"my family, my dog, italy, words and music!",writing my book.,"running with my dog, finishing up the work wee...",i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...
59942,24,single,m,straight,fit,mostly anything,often,sometimes,working on college/university,"white, other",...,i'm nick. i never know what to write about mys...,currently finishing school for film production...,"filmmaking, photography, graphic design, web d...","dude, i don't know.","movies: hook (the greatest adventure ever!), g...",iphone contact lenses headphones camera tv rem...,i do most of my thinking on the bus to/from wo...,"bringin' home bacon, or drinking and shakin'!",when i was 18 i got a tattoo of waldo somewher...,meh if you made it this far you might as well.
59943,42,single,m,straight,average,mostly anything,not at all,never,graduated from masters program,asian,...,"hello! i enjoy traveling, watching movies, and...","i'm a civil engineer, who enjoys helping the c...",- looking at things objectively - getting thin...,i'm quiet until i get used to the environment ...,"last book: ""game change"". movies

In [3]:
embedding_series = pd.read_csv('./embedding_series.csv').set_index('Unnamed: 0')
embedding_series

,embedding
Unnamed: 0,
55435,"[0.020514823496341705, -0.09198534488677979, 0..."
31346,"[-0.03974509984254837, -0.09139120578765869, -..."
23575,"[0.018866078928112984, -0.0564219132065773, 0...."
37715,"[0.11587301641702652, -0.05597995966672897, 0...."
58384,"[0.002988251158967614, -0.07680206745862961, 0..."
...,...
43905,"[0.08724702894687653, -0.07293721288442612, 0...."
33153,"[0.000331356655806303, -0.07900834828615189, 0..."
7678,"[-0.02853960171341896, -0.014243010431528091, ..."


In [152]:
# new_input_matches = rank_new_input("I like bacon and pumpkin pie",'f', 30,40, 0.2)
#returns list of tuples, possibly as strings for some weird reason
#TEMPORARY USE WHILE BUILDING CONSTRUCT_PROMPT. 
matches = pd.read_csv('./okcupid_matches.csv').set_index('Unnamed: 0')

new_input_matches = matches.loc[55435]

In [153]:
!pip install ast
from ast import literal_eval
new_input_matches = literal_eval(new_input_matches['matches'])
new_input_matches

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/0c/ytgqby892k1dcqcnn5gw20f00000gn/T/pip-install-qb_355ty/ast_c646a82fc6b34059bf43387d83fad06a/setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<frozen codecs>", line 906, in open
      FileNotFoundError: [Errno 2] No such file or directory: '/private/var/folders/0c/ytgqby892k1dcqcnn5gw20f00000gn/T/pip-install-qb_355ty/ast_c646a82fc6b34059bf43387d83fad06a/AST/README'
      [end of output]
  
  note: This error originates from a

[(57837, 0.7229513275538625),
 (32304, 0.720553979275116),
 (41271, 0.7122389466715242),
 (55723, 0.709845422683183),
 (24577, 0.7035214856643491),
 (24856, 0.6980997502248912),
 (35073, 0.6934971323052491),
 (28629, 0.6934001708037837),
 (35770, 0.6870449508390966),
 (53961, 0.6858659544974753),
 (39454, 0.6849265506596943),
 (16522, 0.6840750659184236),
 (7566, 0.683595228220249),
 (45458, 0.6816629166865934),
 (10579, 0.680168197082343),
 (44456, 0.6800201338679456),
 (1024, 0.6791760733160084),
 (34206, 0.6791658206279321),
 (37079, 0.677791299052506),
 (23705, 0.6776834040308605),
 (36446, 0.6771668823965459),
 (4833, 0.6765438008229576),
 (12738, 0.6760975057045089),
 (26765, 0.6759330847577657),
 (10691, 0.6758259416378812),
 (24471, 0.6744851405268925),
 (17928, 0.6742697746069406),
 (31221, 0.673574951323189),
 (1038, 0.673137020306301),
 (53875, 0.6723775170263808),
 (28240, 0.6718561919399977),
 (20197, 0.6715672368087098),
 (14038, 0.6702798080782724),
 (51290, 0.6700055483

In [175]:
def construct_prompt(input_string, matches, num_char = False):
    slice_len = min(2, len(matches))
    top_matches_slice = matches[:slice_len]
    essays_to_use = ["essay0", "essay1", "essay2", "essay3", "essay4", 
                   "essay5", "essay6", "essay7", "essay8"]
    prompt = 'Write a dating profile that would be a good match for the input person ' + "Input: " + input_string
    for i, val in top_matches_slice:
        essays_subset = df_all.loc[i,essays_to_use]
        output = essays_subset.str.cat()
        if num_char:
            output = output[:num_char]
        prompt = prompt  + f" Top Match {i + 1}: " + output
    return prompt + f"Input: {input_string}" + " New Profile: "
    

In [176]:
input_str = """Hey there! I'm Jimmy, but you can call me Jim. 
I'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, 
but by night, I'm a dreamer exploring the wonders of the world, both near and far."""
prompt = construct_prompt(input_str, new_input_matches, 200) 
prompt_full = construct_prompt(input_str, new_input_matches) 
prompt

"Write a dating profile that would be a good match for the input person Input: Hey there! I'm Jimmy, but you can call me Jim. \nI'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, \nbut by night, I'm a dreamer exploring the wonders of the world, both near and far. Top Match 57838: i'm addicted to caffeine. i collect random old photographs. i am a master procrastinator and a pop culture junkie. i love philip marlowe. i am currently going through a knitting phase, although someti Top Match 32305: a large portion of my life is consumed by sarcastic jokes, photography, web design, and over thinking simple situations. the other 10% is spent untangling my headphones, cleaning clothes in my never eInput: Hey there! I'm Jimmy, but you can call me Jim. \nI'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, \nbut by night, I'm a dreamer exploring the wonders of the world, both near and far. New Profile: 

In [164]:
generator = pipeline('text-generation', model='gpt2')
all_returned = generator(prompt, do_sample=True, temperature = 0.9, truncation = True,
                         min_length=200, max_length = 1000, num_return_sequences=1)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [165]:
all_returned2 = generator(prompt_full, do_sample=True, temperature = 0.9, truncation = True,
                         min_length=200, max_length = 1000, num_return_sequences=1)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/amaribauer/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 1000, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [166]:
print(all_returned)


[{'generated_text': "Write a dating profile that would be a good match for the input person Input: Hey there! I'm Jimmy, but you can call me Jim. \nI'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, \nbut by night, I'm a dreamer exploring the wonders of the world, both near and far. Top Match 57838: i'm addicted to caffeine. i collect random old photographs. i am a master procrastinator and a pop culture junkie. i love philip marlowe. i am currently going through a knitting phase, although someti Top Match 32305: a large portion of my life is consumed by sarcastic jokes, photography, web design, and over thinking simple situations. the other 10% is spent untangling my headphones, cleaning clothes in my never eInput: Hey there! I'm Jimmy, but you can call me Jim. \nI'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, \nbut by night, I'm a dreamer exploring the wonders of the world, both near an

In [167]:
print(all_returned2)

[{'generated_text': "Write a dating profile that would be a good match for the input person Input: Hey there! I'm Jimmy, but you can call me Jim. \nI'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, \nbut by night, I'm a dreamer exploring the wonders of the world, both near and far. Top Match 57838: i'm addicted to caffeine. i collect random old photographs. i am a master procrastinator and a pop culture junkie. i love philip marlowe. i am currently going through a knitting phase, although sometimes i prefer to sew or crochet or cross stitch. i enjoy ridiculously cheesy movies. i am usually reading at least two books at the same time, and sometimes more (right now: four).  i have been called snarky by more than one person (by more than two, even).  and i really hate having to describe myself.well, there's work (at a publishing company) and other work (freelance), but then, work is not really what defines a life. so what i'm really doing w

In [170]:
output_str = all_returned[0]['generated_text'].replace(prompt, "")
#This is just the generated essay.
#THIS IS WHAT NEEDS TO BE CHECKED FOR TOXICITY
output_str

"\nCurrent profile: \nI'm Jimmy, but you can call me Jim. \nI'm a curious soul with a zest for life and a passion for adventure. -\n-\nI love philip marlowe. \nI work hard as a coder but I enjoy it's hard work and I'm always learning. The more time I spend with a girl I am able to become a better person, the more I like and love her.\n---\nI'm a curious soul with a zest for life and a passion for adventure. -\n-\n-\n- I love philip marlowe. \nI work hard as a coder but I enjoy it's hard work and I'm always learning. The more time I spend with a girl I am able to become a better person, the more I like and love her. New Profile:  \nCurrent profile: \nI'm Jimmy, but you can call me Jim. \nI'm a curious soul with a zest for life and a passion for adventure. -\n-\nI love philip marlowe. \nI work hard as a coder but I enjoy it's hard work and I'm always learning. The more time I spend with a girl I am able to become a better person, the more I like and love her. New Profile:  \nCurrent prof

In [127]:
#BUILD OUT EVALUATION METHODOLOGY WITH RANKING NEW INPUT FUNCTION